In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
from pathlib import Path
import warnings
import os
import random
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
from collections import deque

warnings.simplefilter('ignore')

In [ ]:
def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    
fix_all_seeds(0)

In [ ]:
SAVE_DF = True
SAVE_DF_DIR = Path("/content/drive/MyDrive/Kaggle/BlueCarbon/proc/FE_20230417")

# データ読み込み

In [ ]:
train = pd.read_pickle("/content/drive/MyDrive/Kaggle/BlueCarbon/proc/train.pkl")
test = pd.read_pickle("/content/drive/MyDrive/Kaggle/BlueCarbon/proc/test.pkl")

# 特徴抽出

## MEDの平均と標準偏差抽出

In [ ]:
# clipのパーセンタイル
low_lim = 0.02

# Landsatの列名抽出
cols_landsat = train.columns[313:-1]
# MEDだけ抽出
cols_landsat_med = [col for col in cols_landsat if "MED" in col]

dict_mean_and_std = {}
for col in tqdm(cols_landsat):
    tmp_train = train[col]
    # 最初にclipする
    p01 = tmp_train.quantile(low_lim)
    p99 = tmp_train.quantile(1-low_lim)
    tmp_train = np.clip(tmp_train, p01, p99)

    tmp_dict = {
        "mean": np.nanmean(tmp_train),
        "std": np.nanstd(tmp_train)
    }

    dict_mean_and_std[col] = tmp_dict
    
dict_mean_and_std["MED_Blue_2010"]

  0%|          | 0/3150 [00:00<?, ?it/s]

{'mean': 56.141246535667385, 'std': 8.666390990148283}

## 標準化

In [ ]:
# train
np_year = train["year"].values
cols_landsat = train.columns.to_list()[25:82]

# 各年のデータがない列
cols_nonexist = ["MSAVI", "NBR", "NBR2", "NDMI", "NDVI", "NDWI", "SAVI"]
cols_landsat = [col for col in cols_landsat if col not in cols_nonexist]

isFirstOne = True
for col in tqdm(cols_landsat):
    np_values = train[col].values
    dq_values_normed = deque()
    for year, value in zip(np_year, np_values):
        year = int(year)
        if year < 2000:
            year = 2000
        key_name = f"MED_{col}_{int(year)}"
        mean = dict_mean_and_std[key_name]["mean"]
        std = dict_mean_and_std[key_name]["std"]
        value_normed = (value - mean) / std
        dq_values_normed.append(value_normed)
        
    if isFirstOne:
        isFirstOne = False
        array_normalized_train = np.array(dq_values_normed)
    else:
        array_normalized_train = np.vstack((array_normalized_train, np.array(dq_values_normed)))

cols_landsat_mod = [f"{col}_norm" for col in cols_landsat]
df_landsat_normalized_train = pd.DataFrame(array_normalized_train.T, columns=cols_landsat_mod)
df_landsat_normalized_train

  0%|          | 0/50 [00:00<?, ?it/s]

,Blue_norm,Green_norm,Red_norm,NIR_norm,SWIR1_norm,TIRS1_norm,TIRS2_norm,SWIR2_norm,EVI_norm,TSAVI_norm,...,NLI_norm,NormG_norm,NormR_norm,PPR_norm,PSNDc2_norm,RDVI_norm,IF_norm,SLAVI_norm,SIPI2_norm,VARIgreen_norm
0,-0.872641,-0.770192,-0.624147,-0.536028,-0.546227,-0.744897,-0.828209,-0.532998,-0.085934,-0.914298,...,-1.451049,0.475077,0.296024,-1.109615,-0.750047,-0.052587,0.127271,-0.406854,0.116049,-0.049049
1,0.475056,1.254295,1.274292,3.491892,1.998576,0.583671,0.600471,1.090840,-9.960450,5.201679,...,1.864528,-3.654105,-1.126482,2.711313,5.444383,9.581726,12.557637,1.335754,-4.261068,0.143220
2,-0.528401,-0.312142,-0.227745,-0.315141,-0.386470,-0.168513,-0.217690,-0.471045,0.696263,-0.476337,...,0.035486,-0.117236,0.834750,0.516590,-0.066531,-0.639787,0.237371,-0.263672,-1.031771,1.119617
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-0.252609,-0.487568,-0.623296,-0.518958,-0.564338,0.047077,0.025840,-0.606034,-1.297505,-0.176624,...,-0.783509,1.893325,-3.121324,-1.107131,-1.045281,1.074240,-0.438738,-0.309610,3.230766,-0.070036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14135,-0.764995,-0.681721,-0.565879,-0.332391,-0.342981,-0.926598,-0.815282,-0.438274,-1.096631,1.740217,...,0.843478,-0.546332,-1.052425,-0.797835,0.331930,1.954570,0.156625,-0.386479,6.677195,-0.195885
14136,0.700272,0.970412,0.379811,-0.216631,-0.135309,0.183624,0.103544,-0.108964,1.662544,-1.940657,...,-0.211792,0.750751,0.473771,1.571697,-0.548151,-4.329146,3.155756,-0.192562,-1.593162,0.384475
14137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# test
np_year = test["year"].values
# cols_landsat = train.columns.to_list()[25:82]

# 各年のデータがない列
# cols_nonexist = ["MSAVI", "NBR", "NBR2", "NDMI", "NDVI", "NDWI", "SAVI"]
# cols_landsat = [col for col in cols_landsat if col not in cols_nonexist]

isFirstOne = True
for col in tqdm(cols_landsat):
    np_values = test[col].values
    dq_values_normed = deque()
    for year, value in zip(np_year, np_values):
        year = int(year)
        if year < 2000:
            year = 2000
        key_name = f"MED_{col}_{int(year)}"
        mean = dict_mean_and_std[key_name]["mean"]
        std = dict_mean_and_std[key_name]["std"]
        value_normed = (value - mean) / std
        dq_values_normed.append(value_normed)
        
    if isFirstOne:
        isFirstOne = False
        array_normalized_test = np.array(dq_values_normed)
    else:
        array_normalized_test = np.vstack((array_normalized_test, np.array(dq_values_normed)))

df_landsat_normalized_test = pd.DataFrame(array_normalized_test.T, columns=cols_landsat_mod)
df_landsat_normalized_test

  0%|          | 0/50 [00:00<?, ?it/s]

,Blue_norm,Green_norm,Red_norm,NIR_norm,SWIR1_norm,TIRS1_norm,TIRS2_norm,SWIR2_norm,EVI_norm,TSAVI_norm,...,NLI_norm,NormG_norm,NormR_norm,PPR_norm,PSNDc2_norm,RDVI_norm,IF_norm,SLAVI_norm,SIPI2_norm,VARIgreen_norm
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,-0.108190,-0.679711,-1.012410,-0.993365,-0.865615,1.512704,1.264661,-1.055802,-1.223607,-1.236722,...,-1.565489,1.711719,-1.808100,-1.470230,-1.411384,-0.255966,-0.747009,-0.834219,1.539158,-0.019142
4,-1.039903,-0.526969,-0.126538,-0.555339,-0.712833,-1.295499,-1.038941,-0.780245,1.107904,-1.358693,...,-1.406469,0.020606,1.665615,0.403374,-0.527828,-1.131057,-0.183290,-0.427288,-1.107269,1.075433
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4034,-1.067905,-0.863556,-0.616128,-0.489424,-0.715883,-1.098626,-1.741705,-0.759164,0.092352,-0.404571,...,-0.429043,0.227467,0.233368,-0.411278,-0.385826,-0.088546,-0.450957,-0.453798,-0.104738,-7.308475
4035,-1.041258,-1.037614,-0.752035,-0.692037,-0.596888,-0.394532,-0.432306,-0.552555,-0.140983,-0.450447,...,-0.567387,0.095144,0.413254,-0.936163,-0.521291,0.107754,-0.989161,-0.683312,-0.034884,-0.115609
4036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4037,0.497325,0.293131,0.255721,0.060993,0.052815,0.889662,0.795072,0.130056,0.553273,0.505342,...,1.062942,-0.782298,1.086018,0.414175,0.468529,-0.536797,0.270187,-0.202065,-0.669298,0.415259


In [ ]:
test[cols_landsat]

,Blue,Green,Red,NIR,SWIR1,TIRS1,TIRS2,SWIR2,EVI,TSAVI,...,NLI,NormG,NormR,PPR,PSNDc2,RDVI,IF,SLAVI,SIPI2,VARIgreen
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,55.203629,28.700424,11.296329,4.164535,0.284744,8.604989,8.568386,0.033908,0.052273,-0.711587,...,0.211141,0.649900,0.255797,-0.315875,-0.859705,-1.813769,2.313358,47.185164,7.156558,-1.144489
4,56.845776,42.104111,24.134905,5.699820,0.479576,8.590875,7.937538,0.096436,0.167693,-0.928132,...,0.147521,0.585277,0.335492,-0.148981,-0.817739,-3.375080,3.437880,138.114280,2.774381,1.912993
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4034,56.495659,37.095886,18.232040,6.494017,0.474833,8.619898,7.849435,0.103544,0.095639,-0.731478,...,0.396333,0.600044,0.294912,-0.207281,-0.793807,-2.360574,2.944749,119.071587,4.259801,-16.154256
4035,47.117294,25.835388,12.801763,5.477726,0.447251,8.325656,8.321134,0.108642,0.067792,-0.631541,...,0.401888,0.585639,0.290192,-0.291722,-0.791702,-1.713044,2.224855,70.719658,5.685330,-1.536958
4036,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4037,100.600929,74.113968,47.805202,28.081585,4.378788,8.763263,8.736059,1.153947,0.112425,-0.449125,...,0.885686,0.494091,0.318700,-0.151601,-0.563552,-2.264141,2.986545,1374.850518,3.676777,1.234096


In [ ]:
if SAVE_DF:
    df_landsat_normalized_train.to_pickle(SAVE_DF_DIR / "20230430_train_landsat3_normalized.pkl")
    df_landsat_normalized_test.to_pickle(SAVE_DF_DIR / "20230430_test_landsat3_normalized.pkl")